# Spending a P2SH(Multisig) - BX Exercise
In this exercise, we spend a p2sh(multisig) input to a p2pkh address.

<hr style="border: 0.5px dashed #000;">

## 1. Restore wallet and generate spending key pairs.


In [ ]:
## my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="seven mail crash you unit small assume express wedding cloud work potato"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)


### Retore the three multisig key pairs from our 3rd wallet account.

We will be spending a 2-of-3 output script controlled by these keys.

* `m/44'/1'/2'/0/0`
* `m/44'/1'/2'/0/1`
* `m/44'/1'/2'/0/2`

In [ ]:
hd_m_44h_1h_2h_0_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0 \
| bx hd-private --index 0)
privatekey_44h_1h_2h_0_0=$(bx hd-to-ec $hd_m_44h_1h_2h_0_0)
publickey_44h_1h_2h_0_0=$(bx hd-to-public $hd_m_44h_1h_2h_0_0 | bx hd-to-ec)

hd_m_44h_1h_2h_0_1=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0 \
| bx hd-private --index 1)
privatekey_44h_1h_2h_0_1=$(bx hd-to-ec $hd_m_44h_1h_2h_0_1)
publickey_44h_1h_2h_0_1=$(bx hd-to-public $hd_m_44h_1h_2h_0_1 | bx hd-to-ec)

hd_m_44h_1h_2h_0_2=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0 \
| bx hd-private --index 3)
privatekey_44h_1h_2h_0_2=$(bx hd-to-ec $hd_m_44h_1h_2h_0_2)
publickey_44h_1h_2h_0_2=$(bx hd-to-public $hd_m_44h_1h_2h_0_2 | bx hd-to-ec)

echo $publickey_44h_1h_2h_0_0
echo $publickey_44h_1h_2h_0_1
echo $publickey_44h_1h_2h_0_2

### Fetch previous UTXO's you are spending.

In [ ]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index]"

bx fetch-tx c984d7c2cd330f8a980a8953f3b6141b6441933054838eb09c29f1709775da7c -f json | jq ".transaction.outputs[0]"

In [ ]:
# previous_txid=
# previous_output_index=
# previous_output_amount=

previous_txid=c984d7c2cd330f8a980a8953f3b6141b6441933054838eb09c29f1709775da7c
previous_output_index=0
previous_output_amount=499360

## 2. Sign input and output.

### A. Encode p2pkh output script.
* Please spend testnet coins to p2pkh address `n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw`


In [ ]:
# Decode address for public key hash.
bx address-decode n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw

* Encode p2pkh(`n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw`) script.

In [ ]:
# output_script=

output_script=$(bx script-encode "DUP HASH160 [e48199d47742b245464b1366d95ef26aa4c8bb2c] EQUALVERIFY CHECKSIG")
echo $output_script


### B. Compute output value

* **Subtract minimum fee, which consists of:**
    * Example minimum fee levels: `100 sat / sigop + 1 sat / byte`
    * This is node policy. 


* **Transaction sigop count computation:**
    * Counted as 1 sigop: `checksig`,`checksigverify`
    * Counted as 20 sigops: `checkmultisig`, `checkmultisigverify`
    * *Non-segwit:*
        * Transaction sigop count is multiplied by 4x.
        * Output vs Input sigops: 
            * All sigops in the output script are counted.
            * P2SH Redeemscript input sigops are counted.
      

In [ ]:
# tx bytes: 4 + 1 + 1*(32+4+(1+72+72+(1+23))+4) + 1 + 1*(8+1+25) + 4 

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 1 \* 209 + 1 + 1 \* 34 + 4)
echo $tx_byte_count


In [ ]:
# Output amount total.

# output_amount=

output_amount=$(expr $previous_output_amount - 20 \* 400 - $tx_byte_count \* 1)
echo $output_amount


### C. Construct Transaction


In [ ]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# my_tx=

my_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $output_script:$output_amount)

echo $my_tx | bx tx-decode

### D. Sign transaction & finalise transaction.

**Fetch previous output script.**

In [ ]:
# Common Gotcha. Previous output script not previous p2sh output, but p2sh embedded script.

# previous_output_script=

previous_output_script="2 [03a151f572ab9462701f055904c7e80e12dd64cce5e59368e741139415a5e129cd] [0293798c3c194caf03cd12edb23351a94a69706638e712fadd3abaa0db791921a1] [02035765ecd6c51d151d3676bd2f87307030a0d5d3f32a4636707b345a444b074e] 3 checkmultisig"

echo $previous_output_script | bx script-encode

**Sign transaction with private keys ``**

* private key `/44'/1'/2'/0/0`
* private key `/44'/1'/2'/0/1`
* private key `/44'/1'/2'/0/2`

In [ ]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature0=
# signature1=
# signature2=

signature0=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_2h_0_0 "$previous_output_script" $my_tx)
signature1=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_2h_0_1 "$previous_output_script" $my_tx)
signature2=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_2h_0_2 "$previous_output_script" $my_tx)


**Set input into finalised transaction**

In [ ]:
# bx input-set "0 [signature0] [signature1] [multisig_script]" transaction

# my_tx=

my_tx=$(bx input-set --index 0 "zero [$signature1] [$signature2] [522103a151f572ab9462701f055904c7e80e12dd64cce5e59368e741139415a5e129cd210293798c3c194caf03cd12edb23351a94a69706638e712fadd3abaa0db791921a12102035765ecd6c51d151d3676bd2f87307030a0d5d3f32a4636707b345a444b074e53ae]" $my_tx)
echo $my_tx | bx tx-decode


In [ ]:
bx validate-tx $my_tx


**What combinations of signatures are valid?**